In [ ]:
import os
import smtpd
import shutil
import socket
import tkinter
import asyncore
from tkinter import *
from smtpd import SMTPServer
from datetime import datetime
from tkinter import messagebox
from tkinter.messagebox import showerror

root = tkinter.Tk()
root.resizable(False, False)
root.title('SMTP EMAIL SERVER')
ws = root.winfo_screenwidth()
hs = root.winfo_screenheight()
w = 992
h = 550
x = (ws/2) - (w/2)
y = (hs/2) - (h/2)
root.geometry('%dx%d+%d+%d' % (w, h, x, y))

user_names = []

users = []

class EmlServer(SMTPServer):
    no = 0
        
    def process_message(self, peer, mailfrom, rcpttos, data, mail_options=None,rcpt_options=None):
        message = str(data).split('\\n')
        userName = (message[6].replace('username:','')).strip()
        passWord = (message[7].replace('password:','')).strip()
        receiver_temp = (message[5].replace('To:','')).strip()
        receivers = receiver_temp.split(',')
        for i in receivers:
            receiver = (i.replace('@dcnproject.com','')).strip()
            message_body = message[3]
            message_body += '\n' + message[4]
            message_body += '\n' + message[5]
            message_body += '\n\n' + message[-1]
            if validate_sender(userName,passWord):
                if validate_receiver(receiver):
                    filename = f'{receiver}/received/'
                    filename += '%s-%d.txt' % (datetime.now().strftime('%Y%m%d%H%M%S'), self.no)
                    f = open(filename, 'w')
                    f.write(message_body)
                    f.close()

                    filename = f'{userName}/sent/'
                    filename += '%s-%d.txt' % (datetime.now().strftime('%Y%m%d%H%M%S'), self.no)
                    f = open(filename, 'w')
                    f.write(message_body)
                    f.close()
                else:
                    showerror('Error','Receiver address not found')
            else:
                showerror('Error','Sender could not be verified')
                
        
def validate_sender(username, password):
    global users
    for i in users:
        if [username,password] == i:
            return True
            break
            
    return False
    
def validate_receiver(receiver):
    global user_names
    for i in user_names:
        if receiver == i:
            return True
            break
    
    return False
    
server = EmlServer(('localhost', 25), None)

def start_server():
    global users
    if len(users):
        asyncore.loop()
    else:
        showerror('Error','Server can not run wothout users')
    
def validate():
    try:
        global user_names
        if user_name.get() and pass_word.get():
            if (user_name.get()).lower() in user_names:
                showerror('Error','User already exists')
                return
            if len(user_name.get()) < 3:
                showerror('Error','Username must be greater than or equal to 3 characters')
                return
            if not user_name.get().isalnum():
                showerror('Error','Username can be only alphanumeric')
                return
            users.append([user_name.get(),pass_word.get()])
            write_users()
            os.mkdir(f'{user_name.get()}')
            os.mkdir(f'{user_name.get()}/sent')
            os.mkdir(f'{user_name.get()}/received')
            user_input.delete(0, END)
            pass_input.delete(0, END)
            user_input.focus()
        else:
            showerror('Error','Both username and password is required')
    except:
        showerror('Sorry','Something went wrong')

def selection(event):
    details = users_listBox.get(users_listBox.curselection())
    user_name.set(details[0])
    pass_word.set(details[1])
    del_btn.config(state=NORMAL)
    
def delete_user():
    global users
    global user_names
    
    if user_name.get() == '':
        showerror('Error','Enter username first')
        return
    if user_name.get() not in user_names:
        showerror('Error','User not found')
        return
    result = messagebox.askquestion('Confirmation', 'Are You Sure?', icon='warning')
    if result == 'yes':
        for i in range(len(users)):
            if user_name.get() ==  users[i][0]:
                users.pop(i)
                break
        shutil.rmtree(user_name.get())
        user_input.delete(0,END)
        pass_input.delete(0,END)
        del_btn.config(state=DISABLED)
        write_users()
    
top_frame = Frame(root, width=992, height=100, bg='#0f4c81')
top_frame.pack(side=TOP)
top_frame.pack_propagate(0)

heading_label = Label(top_frame, width=1000, height=100, bg='#0f4c81', fg='#fff',text='SMTP EMAIL SERVER', font=('Arial',24))
heading_label.pack(side=TOP)
heading_label.config(anchor=CENTER)

left_frame = Frame(root, width=720, height=450, bg='#eee')
left_frame.pack(side=LEFT)
left_frame.pack_propagate(0)

sub_heading_label = Label(left_frame, bg='#eee', fg='#1b262c',text='EMAIL ACCOUNTS', font=('Arial',16))
sub_heading_label.pack(side=TOP, pady=(20,0))
sub_heading_label.config(anchor=CENTER)

users_listBox = Listbox(left_frame, width=720, height=450, font=('Arial',12))
users_listBox.pack(side=TOP, padx=(20), pady=(15,30))

users_listBox.bind('<<ListboxSelect>>', selection)

right_frame = Frame(root, width=272, height=450, bg='#eee')
right_frame.pack(side=RIGHT)
right_frame.pack_propagate(0)

user_label = Label(right_frame, font=('Arial',12), text='USERNAME:', bg='#eee',fg='#1b262c')
user_label.pack(side=TOP, pady=(65,0), ipady=(5), ipadx=(70))
user_label.config(anchor='w')

user_name = StringVar()

user_input = Entry(right_frame, font=('Arial',10), textvariable=user_name)
user_input.pack(side=TOP, pady=(1,0), ipady=(5), ipadx=(50))

pass_label = Label(right_frame, font=('Arial',12), text='PASSWORD:', bg='#eee',fg='#1b262c')
pass_label.pack(side=TOP, pady=(10,0), ipady=(5), ipadx=(70))
pass_label.config(anchor='w')

pass_word = StringVar()

pass_input = Entry(right_frame, font=('Arial',10), textvariable=pass_word)
pass_input.pack(side=TOP, pady=(1,0), ipady=(5), ipadx=(50))

add_btn = Button(right_frame, font=('Arial',10), text='ADD USER', bg='#1b262c', fg='#fff', command=validate)
add_btn.pack(side=TOP, fill=X, padx=(15), pady=(10,0), ipady=(5))

del_btn = Button(right_frame, font=('Arial',10), text='DELETE USER', bg='#1b262c', fg='#fff', command=delete_user)
del_btn.pack(side=TOP, fill=X, padx=(15), pady=(10,0), ipady=(5))
del_btn.config(state=DISABLED)

start_btn = Button(right_frame, font=('Arial',10), text='START SERVER', bg='#1b262c', fg='#fff', command=start_server)
start_btn.pack(side=TOP, fill=X, padx=(15), pady=(30,0), ipady=(5))

def read_users():
    file = open('users.txt','r')
    global users
    global users_names
    lines = file.read().splitlines()
    file.close()
    if len(lines):
        for line in lines:
            if line != '':
                users.append(line.split('|'))
        for i in range(len(users)):
            user_names.append(users[i][0])
            users_listBox.insert(END,users[i])
            
def write_users():
    global users
    global user_names
    users_listBox.delete(0,END)
    user_names = []
    f = open('users.txt','w+')
    if len(users):
        for i in range(len(users)):
            user_names.append(users[i][0])
            f.write(f'{users[i][0]}|{users[i][1]}\n')
            users_listBox.insert(END,users[i])
    f.close()
            
read_users()

root.mainloop()